In [11]:
import pandas as pd
from pathlib import Path
import re
from io import StringIO
import sys

In [12]:
_ROOT = Path.cwd().parent 

In [13]:
sys.path.append(_ROOT.as_posix())
from common.s3_service import S3Connector
s3 = S3Connector(_ROOT)

In [14]:
files = s3.list_files_in_s3("gpt-responses")

In [18]:
df_texto = s3.read_csv_from_s3("gpt-responses", files[0])

In [19]:
hay_duplicados = len(df_texto["url"]) != len(set(df_texto["url"]))

if hay_duplicados:
    print("Hay duplicados en la lista de URLs")
else:
    print("No hay duplicados en la lista de URLs")

No hay duplicados en la lista de URLs


In [290]:
ids = [i+ 1 for i in range(len(df_texto["url"]))]

In [291]:
# Agregar la lista de IDs como una nueva columna
df_texto["id_url"] = ids

In [292]:
df_texto["image_response_split"] = df_texto["image_response"].str.split(";")

In [293]:
df_texto = df_texto.explode("image_response_split")

In [294]:
df_texto = df_texto.reset_index()

In [295]:
categorias = ["Número de piezas", "\nDescripción prenda:\n", "\nZapatos", "\nAccesorios"]

In [296]:
def categorizar(fila):
    for categoria in categorias:
        if fila["image_response_split"].startswith(categoria):  # Eliminado .str
            return categoria
    return None

df_texto["categoria"] = df_texto.apply(categorizar, axis=1)

In [297]:
df_texto["categoria"].value_counts()

\nDescripción prenda:\n    44
Número de piezas           29
\nZapatos                  28
\nAccesorios               25
Name: categoria, dtype: int64

In [298]:
diccionario_cat = {
    "\nDescripción prenda:\n": "prendas",
    "Número de piezas": "n piezas",
"\nZapatos": "zapatos",
"\nAccesorios": "accesorios"
}

In [299]:
df_texto["etiqueta"] = df_texto["categoria"].map(diccionario_cat)

In [300]:
df_prendas = df_texto[df_texto["etiqueta"]=="prendas"]
df_accesorios = df_texto[df_texto["etiqueta"]=="accesorios"]
df_zapatos = df_texto[df_texto["etiqueta"]=="zapatos"]
df_n_prendas = df_texto[df_texto["etiqueta"]=="n prendas"]

In [301]:
df_prendas = df_prendas[["id_url","image_response_split"]]
df_zapatos = df_prendas[["id_url","image_response_split"]]
df_accesorios = df_prendas[["id_url","image_response_split"]]
df_n_prendas = df_prendas[["id_url","image_response_split"]]

In [302]:
df_prendas["category"]=df_prendas["image_response_split"].str.split("\n")
df_prendas = df_prendas.explode("category")
df_prendas.head(4)

,id_url,image_response_split,category
1,1,\nDescripción prenda:\n - Tipo: Vestido\n - Co...,
1,1,\nDescripción prenda:\n - Tipo: Vestido\n - Co...,Descripción prenda:
1,1,\nDescripción prenda:\n - Tipo: Vestido\n - Co...,- Tipo: Vestido
1,1,\nDescripción prenda:\n - Tipo: Vestido\n - Co...,- Color: Negro


In [307]:
df_prendas.reset_index(inplace=True)

In [314]:
df_prendas = df_prendas[["id_url","image_response_split","category"]]

In [316]:
nulito = df_prendas["category"].iloc[0]

In [317]:
df_drop = df_prendas[df_prendas["category"]==nulito]

In [321]:
indices_eliminar = df_drop.index

In [325]:
indices_eliminar = indices_eliminar.to_list()

In [326]:
df_prendas = df_prendas.loc[~df_prendas.index.isin(indices_eliminar)]

In [327]:
df_prendas

,id_url,image_response_split,category
1,1,\nDescripción prenda:\n - Tipo: Vestido\n - Co...,Descripción prenda:
2,1,\nDescripción prenda:\n - Tipo: Vestido\n - Co...,- Tipo: Vestido
3,1,\nDescripción prenda:\n - Tipo: Vestido\n - Co...,- Color: Negro
4,1,\nDescripción prenda:\n - Tipo: Vestido\n - Co...,- Material: Vinilo
5,1,\nDescripción prenda:\n - Tipo: Vestido\n - Co...,- Categoría: Mujer
...,...,...,...
639,29,\nDescripción prenda:\n - Tipo: Top\n - Color:...,- Mangas: Cortas
640,29,\nDescripción prenda:\n - Tipo: Top\n - Color:...,- Cuello: Redondo
641,29,\nDescripción prenda:\n - Tipo: Top\n - Color:...,- Escote: N/A
642,29,\nDescripción prenda:\n - Tipo: Top\n - Color:...,- Largo: Corto


In [328]:
df_prendas["data"]=df_prendas["category"].str.split(":")
df_prendas.reset_index()
df_prendas.head(1)

,id_url,image_response_split,category,data
1,1,\nDescripción prenda:\n - Tipo: Vestido\n - Co...,Descripción prenda:,"[Descripción prenda, ]"


In [329]:
# Dividir el texto en dos columnas por ":"
df_prendas[["feature", "information"]] = df_prendas["data"].str.split(":", 1, expand=True)

ValueError: Columns must be same length as key

0

In [330]:
df_prendas.head(10)

,id_url,image_response_split,category,data
1,1,\nDescripción prenda:\n - Tipo: Vestido\n - Co...,Descripción prenda:,"[Descripción prenda, ]"
2,1,\nDescripción prenda:\n - Tipo: Vestido\n - Co...,- Tipo: Vestido,"[ - Tipo, Vestido]"
3,1,\nDescripción prenda:\n - Tipo: Vestido\n - Co...,- Color: Negro,"[ - Color, Negro]"
4,1,\nDescripción prenda:\n - Tipo: Vestido\n - Co...,- Material: Vinilo,"[ - Material, Vinilo]"
5,1,\nDescripción prenda:\n - Tipo: Vestido\n - Co...,- Categoría: Mujer,"[ - Categoría, Mujer]"
6,1,\nDescripción prenda:\n - Tipo: Vestido\n - Co...,- Diseño: Ajustado,"[ - Diseño, Ajustado]"
7,1,\nDescripción prenda:\n - Tipo: Vestido\n - Co...,- Textura: Brillante,"[ - Textura, Brillante]"
8,1,\nDescripción prenda:\n - Tipo: Vestido\n - Co...,- Estampado: Ninguno,"[ - Estampado, Ninguno]"
9,1,\nDescripción prenda:\n - Tipo: Vestido\n - Co...,- Mangas: Cortas,"[ - Mangas, Cortas]"
10,1,\nDescripción prenda:\n - Tipo: Vestido\n - Co...,- Cuello: Babero,"[ - Cuello, Babero]"


In [331]:
categorias_prendas = ["- Tipo:","- Color:","- Material:","- Categoría:","- Diseño:","- Textura:","- Estampado:","- Mangas:","- Cuello:","- Escote:","- Largo:","- Detalles:"]

In [332]:
df_prendas.reset_index(drop=True)

,id_url,image_response_split,category,data
0,1,\nDescripción prenda:\n - Tipo: Vestido\n - Co...,Descripción prenda:,"[Descripción prenda, ]"
1,1,\nDescripción prenda:\n - Tipo: Vestido\n - Co...,- Tipo: Vestido,"[ - Tipo, Vestido]"
2,1,\nDescripción prenda:\n - Tipo: Vestido\n - Co...,- Color: Negro,"[ - Color, Negro]"
3,1,\nDescripción prenda:\n - Tipo: Vestido\n - Co...,- Material: Vinilo,"[ - Material, Vinilo]"
4,1,\nDescripción prenda:\n - Tipo: Vestido\n - Co...,- Categoría: Mujer,"[ - Categoría, Mujer]"
...,...,...,...,...
552,29,\nDescripción prenda:\n - Tipo: Top\n - Color:...,- Mangas: Cortas,"[ - Mangas, Cortas]"
553,29,\nDescripción prenda:\n - Tipo: Top\n - Color:...,- Cuello: Redondo,"[ - Cuello, Redondo]"
554,29,\nDescripción prenda:\n - Tipo: Top\n - Color:...,- Escote: N/A,"[ - Escote, N/A]"
555,29,\nDescripción prenda:\n - Tipo: Top\n - Color:...,- Largo: Corto,"[ - Largo, Corto]"


In [333]:
def categorizar_prenda(fila):
  for categoria in categorias_prendas:
    if fila["category"].startswith(categoria):
      return categoria
  return None

df_prendas["test"] = df_prendas.apply(categorizar_prenda, axis=1)


In [334]:
df_prendas

,id_url,image_response_split,category,data,test
1,1,\nDescripción prenda:\n - Tipo: Vestido\n - Co...,Descripción prenda:,"[Descripción prenda, ]",None
2,1,\nDescripción prenda:\n - Tipo: Vestido\n - Co...,- Tipo: Vestido,"[ - Tipo, Vestido]",None
3,1,\nDescripción prenda:\n - Tipo: Vestido\n - Co...,- Color: Negro,"[ - Color, Negro]",None
4,1,\nDescripción prenda:\n - Tipo: Vestido\n - Co...,- Material: Vinilo,"[ - Material, Vinilo]",None
5,1,\nDescripción prenda:\n - Tipo: Vestido\n - Co...,- Categoría: Mujer,"[ - Categoría, Mujer]",None
...,...,...,...,...,...
639,29,\nDescripción prenda:\n - Tipo: Top\n - Color:...,- Mangas: Cortas,"[ - Mangas, Cortas]",None
640,29,\nDescripción prenda:\n - Tipo: Top\n - Color:...,- Cuello: Redondo,"[ - Cuello, Redondo]",None
641,29,\nDescripción prenda:\n - Tipo: Top\n - Color:...,- Escote: N/A,"[ - Escote, N/A]",None
642,29,\nDescripción prenda:\n - Tipo: Top\n - Color:...,- Largo: Corto,"[ - Largo, Corto]",None
